In [1]:
# Function to convert bounding box to polygon
import pandas as pd
from shapely.geometry import Polygon
import datasets
from shapely.geometry import box

import sys
sys.path.append('../src/utils')

from ac_object_etl_functions import *

In [ ]:
#%load_ext memray
# Load in the data labels
params = {}
#params['category_index_column'] = "present1hot"
params['category_labels'] = ['No', 'Yes']
params['images_folder'] = '../data/images/ac_150_images'
params["data_labels_path"] = '../data/labels/ac_150_labels_jess_notes.csv'
params["annotation_path"] = '../data/labels/Export v2 project - AC Unit Evaluation Test - 7_3_2024.ndjson'
params["iou_threshold"] = 1.0/3
params["annotators_per_image"] = 5
# params['kernal'] = np.ones((12,12), np.uint8)
# params['hsv_range'] = [[8, 120, 200], [12, 255, 255]]
# params['min_ratio'] = 2
params['dataset_version'] = 'v4'
params['rerun_ds_generation'] = True

prime_labels = load_ac_data_spreadsheet(params)
prime_labels = prime_labels.rename(columns = {'row_number ': 'row_number'})
label_set = pd.read_json(params["annotation_path"], lines=True)
df = extract_label_data(label_set)
#join prime_labels row_number to df with image_id
df = pd.merge(df, prime_labels[['image_id','row_number']], on = 'image_id', how = 'inner')
df = df.rename(columns = {'row_number': 'image_orig_idx'})

#remove intersecting bounding box (specific to ../data/images/ac_150)
df = df[df['feature_id'] != 'clxgcq78i00hx3b6qit987ryb']

matched_sets = generate_matched_sets(df, params["iou_threshold"], params["annotators_per_image"])

params["image_ids"] = prime_labels['image_id']
mask_df = mask_images_by_box(params)

#fix bounding boxes (specific to ../data/images/ac_150)
mask_df.loc[mask_df['image_id'] == 'EC33B284-5C10-4EDA-AD17-5571259F97CA.jpeg',['x','y','w','h']] = [480,250,450,550]
mask_df.loc[mask_df['image_id'] == '0AD2D165-CE95-4F5C-BF88-524EA08A686E.jpeg',['x','y','w','h']] = [730,375,218,390]

# turn columns x,y,w,h in shapely polygons
mask_df['mask_geometry'] = mask_df.apply(lambda row: box(row['x'], row['y'], row['x']+row['w'], row['y']+row['h']), axis=1)
mask_df['mask_geometry'] = mask_df['mask_geometry'].apply(lambda x: Polygon(x.exterior.coords))

full_train = pd.merge(matched_sets, mask_df, on = 'image_id', how = 'outer')
no_annotation_rows = full_train.loc[full_train['roi_index'].isna(),:]

no_annotation_rows.loc[no_annotation_rows['roi_index'].isna(),['roi_index','name','num_agree_in_set','set_order','connected_sets','averaged_polygons','percent_agreement']] = [-1,'AC Unit',5,0,{},Polygon([(0,0),(0,0),(0,0),(0,0),(0,0)]),100]

no_annotation_rows2 = no_annotation_rows.copy(deep = True)
no_annotation_rows2['name'] = 'AC leaking'

full_train = pd.concat([full_train.dropna(subset = ['roi_index']), no_annotation_rows, no_annotation_rows2], axis = 0, ignore_index = True)


# full_train.loc[full_train['roi_index'] != -1,'has_roi'] = 1
# full_train.loc[full_train['roi_index'] == -1,'has_roi'] = 0

full_train['mask_bbox'] = full_train['mask_geometry'].apply(lambda x: convert_polygon_to_bbox(x))
full_train['category_bbox'] = full_train['averaged_polygons'].apply(lambda x: convert_polygon_to_bbox(x))


if params['rerun_ds_generation']:
    ds = generate_hf_ds(full_train.dropna(subset = ['roi_index']), mask_df, params)
else:
    print('Dataset generation skipped')
    print('Loading previously generated dataset')
    ds = datasets.load_from_disk("../data/models/ac_model/ac_object_" + params['dataset_version'] + ".hf")

     

In [ ]:
prime_labels = load_ac_data_spreadsheet(params)

In [ ]:
prime_labels = prime_labels.reset_index()
[['image_id','row_number']]

In [ ]:
image_ids = ['EC33B284-5C10-4EDA-AD17-5571259F97CA.jpeg','0AD2D165-CE95-4F5C-BF88-524EA08A686E.jpeg']

for image_id in image_ids:
    patch_df = full_train[(full_train["image_id"] == image_id)].reset_index(drop=True)
    category = []
    mask_geometry = patch_df['mask_geometry'].iloc[0]
    category_polygons = []
    image_path = patch_df['image_path'].iloc[0]
    display_image_with_mask(image_path, mask_geometry, category_polygons, category, zoom = False)

In [ ]:
spot_check_annotations(full_train, 0, 80, False)
spot_check_annotations(full_train, 404, 80, False)
